In [1]:
import faiss

In [2]:
import numpy as np

In [3]:
import re

In [4]:
import os

In [5]:
from langchain.document_loaders import PDFPlumberLoader

In [6]:
from langchain_ollama.llms import OllamaLLM 

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
from langchain_ollama import OllamaEmbeddings


In [9]:
MODEL = OllamaLLM(model="conceptsintamil/tamil-llama-7b-instruct-v0.2")

In [31]:
from sentence_transformers import SentenceTransformer
EMBEDDING_MODEL =  SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

C:\Users\aravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\aravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aravi\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
PDF_STORAGE_PATH = "C:\\Users\\aravi\\Downloads\\pdftake"

In [12]:
dimension = 768
DOCUMENT_VECTOR_DB = faiss.IndexFlatL2(dimension)

In [13]:
doc_store = {}

In [14]:
def save_uploaded_file(uploaded_file, filename):

    file_path = os.path.join(PDF_STORAGE_PATH, filename)
    with open(file_path, "wb") as file:
        file.write(uploaded_file.read())
    return file_path

In [15]:
def load_pdf_documents(file_path):

    document_loader = PDFPlumberLoader(file_path)
    return document_loader.load()

In [16]:
def chunk_documents(raw_documents):

    text_processor = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=150,
        add_start_index=True
    )
    return text_processor.split_documents(raw_documents)

In [37]:
def index_documents(document_chunks):

    print("hi")
    embeddings = np.array(
        EMBEDDING_MODEL.encode([doc.page_content for doc in document_chunks]),
        dtype=np.float32
    )
    print("hi")
    start_id = len(doc_store)  # Start index for new documents
    for i, doc in enumerate(document_chunks):
        doc_store[start_id + i] = doc.page_content  # Store text
    
    DOCUMENT_VECTOR_DB.add(embeddings) 

In [70]:
def find_related_documents(query, k=2):

    query_embedding = np.array(
        EMBEDDING_MODEL.encode([query]),
        dtype=np.float32
    )


    _, indices = DOCUMENT_VECTOR_DB.search(query_embedding, k)

    results = [doc_store[idx] for idx in indices[0] if idx in doc_store]
    return results  

In [71]:
def expand_query(query):

    expansion_prompt = f"""
    Given this short user query: "{query}"
    Expand it in tamil into a more detailed question for better document retrieval.
    """
    return MODEL.invoke(expansion_prompt)

def prompt_template(user_query, context_text):
    return f"""You are an intelligent assistant tasked with answering user queries strictly based on the provided document.

### **Context (Extracted from Document)**
{context_text}

### **User Query**
{user_query}

### **Instructions for Answering**
- Use only the provided document content to generate a response.
- Do **not** introduce any external information or assumptions.
- Extract relevant details from the document and structure them into a **coherent, well-formed response**.
- Ensure the response is **clear and readable**, maintaining natural formatting without unnecessary line breaks (`\n`).
- Do **not** insert extra newlines unless they are essential for meaning or readability.
- Avoid excessive spacing, unnatural paragraph breaks, or disjointed formatting.
- If special characters like `\n` are found in the extracted content, **omit them unless needed for clarity**.
- If no relevant information is available, respond with: "I'm sorry, but I couldn't find an answer in the provided document."
- Do not provide explanations or context beyond what is necessary to answer the query.

### **Final Answer (Ensure clear, natural formatting without unnecessary newlines)**
"""



In [72]:
def generate_answer(user_query):
    e_user_query = expand_query(user_query)
    related_docs = find_related_documents(e_user_query)
    if not related_docs:
        return ["I'm sorry, but I couldn't find an answer in the provided document.", ""]

    context_text = "\n\n".join([doc for doc in related_docs])
    prompt = prompt_template(e_user_query, context_text)
    ai_response1 = MODEL.invoke(prompt)
    ai_response2 = MODEL.invoke(f"response from an AI model {ai_response1} refine the response to clear,concise and meaninglful by without changing or avoiding any information ")
    ai_response1 = re.split(r"</think>\s*", ai_response1, maxsplit=1)[-1].strip()
    ai_response2 = re.split(r"</think>\s*", ai_response2, maxsplit=1)[-1].strip()
    return [ai_response1,ai_response2, context_text]


In [49]:
uploaded_pdf = "C:\\Users\\aravi\\Downloads\\tamil.pdf"
if uploaded_pdf:
    filename = os.path.basename(uploaded_pdf)
    file_path = save_uploaded_file(open(uploaded_pdf, "rb"), filename)
    print(f"PDF saved at: {file_path}")

PDF saved at: C:\Users\aravi\Downloads\pdftake\tamil.pdf


In [50]:
# Load and process document
raw_docs = load_pdf_documents(file_path)


In [51]:
processed_chunks = chunk_documents(raw_docs)

In [52]:
index_documents(processed_chunks)

hi
hi


In [53]:
# User query example
user_input = "தேசியக் கல்விக் கொள்கை 2020 பற்றிய முக்கிய நோக்கங்கள் என்ன?"

In [57]:
ai_response = generate_answer(user_input)

In [27]:
chunks = [doc.page_content for doc in processed_chunks]

In [58]:
ai_response[0]

'The National Education Policy 2020 emphasizes several key objectives related to education in India. One of these goals involves promoting significant expansion and improvement in higher educational institutions for the benefit of students from various backgrounds. The policy also aims at establishing an effective system that will evaluate, recognize, and reward academic excellence across all private universities as well as colleges. Furthermore, it supports the development of human resource (HR) capabilities through its HR department under 4.41.'

In [64]:
user_input = "உலகம் எந்தத் துறையில் விரைவான மாற்றங்களை அடைந்து வருகிறது?"

In [ ]:
ai_response = generate_answer(user_input)

In [61]:
ai_response[0]

'The world appears to be experiencing rapid changes in various fields, including energy, water resources, and food sources as indicated by the given context. These transformations are often accompanied by environmental challenges like increased pollution.'